# torch.autograd를 사용한 자동 미분
- 신경망을 학습할 때 자주 사용하는 알고리즘은 역전파 알고리즘입니다.
- 가중치는 주어진 매개변수에 대한 손실 함수의 변화도(gradient)에 따라 조정 
- 입력 x, 매개변수 w, b 및 손실 함수가 있는 간단한 신경망 구성하여 예제 실습 

In [4]:
import torch 

x = torch.ones(5) # input tensor 
print(x)

y = torch.zeros(3) # expected output
print(y)

w = torch.randn(5, 3, requires_grad = True)
b = torch.randn(3, requires_grad = True) 
print(w)
print(b)
z = torch.matmul(x, w) + b 
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

tensor([1., 1., 1., 1., 1.])
tensor([0., 0., 0.])
tensor([[-0.7518, -1.0970, -2.0728],
        [ 1.1662, -0.0966,  1.6353],
        [ 0.0392, -0.6682,  0.6577],
        [ 0.8471,  1.9305, -0.8474],
        [ 0.0678,  0.0530, -1.5967]], requires_grad=True)
tensor([ 1.3535, -0.4213,  0.0047], requires_grad=True)


In [5]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x000002B1C10BC460>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x000002B1C10DECA0>


## gradient 계산하기
- 신경망에서 매개변수의 가중치를 최적화하려면 매개변수에 대한 손실함수의 도함수(derivative)를 계산해야함 
- x 와 y의 일부 고정값에서 ∂loss/∂w 와 ∂loss/∂b가 필요
- 도함수를 계산하기 위해 loss.backword()를 호출한 다음 w.grad와 b.grad에서 값을 가져옴

In [6]:
loss.backward() 
print(w.grad)
print(b.grad)

tensor([[0.3128, 0.1419, 0.0327],
        [0.3128, 0.1419, 0.0327],
        [0.3128, 0.1419, 0.0327],
        [0.3128, 0.1419, 0.0327],
        [0.3128, 0.1419, 0.0327]])
tensor([0.3128, 0.1419, 0.0327])


- 연산 그래프의 잎(leaf) 노드들 중 requires_grad 속성이 True로 설정된 노드들의 grad 속성만 구할 수 있습니다. 그래프의 다른 모든 노드에서는 변화도가 유효하지 않습니다.
- 성능 상의 이유로, 주어진 그래프에서의 backward를 사용한 변화도 계산은 한 번만 수행할 수 있습니다. 만약 동일한 그래프에서 여러번의 backward 호출이 필요하면, backward 호출 시에 retrain_graph=True를 전달해야 합니다.

## 변화도 추적 멈추기
- 기본적으로, requires_grad=True인 모든 텐서들은 연산 기록을 추적하고 변화도 계산을 지원
- 그러나 모델을 학습한 뒤 입력 데이터를 단순히 적용하기만 하는 경우와 같이 순전파 연산만 필요한 경우, 이러한 추적이나 지원이 필요없을 수 있습니다. 
- 연산 코드를 torch.no_grad() 블록으로 둘러싸서 연산 추적을 멈추는게 가능 

In [8]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

True


In [9]:
with torch.no_grad():
    z = torch.matmul(x, w) + b 
print(z.requires_grad)

False


- detach() 메소드를 사용하여 동일한 결과를 얻음

In [10]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


변화도 추적을 멈춰야 하는 이유
- 신경망의 일부 매개변수를 고정된 매개변수(frozen parameter)로 표시합니다. 
- 이는 사전 학습된 신경망을 미세조정 할 때 매우 일반적인 시나리오입니다.
- 변화도를 추적하지 않는 텐서의 연산이 더 효율적이기 때문에, 순전파 단계만 수행할 때 연산 속도가 향상됩니다.

## 연산 그래프에 대한 추가 정보
- autograd는 데이터 및 실행된 모든 연산들의 기록을 Function 객체로 구성된 방향성 비순환 그래프(DAG)에 저장 
- DAG의 잎은 입력 텐서이고, 뿌리는 결과 텐서 
- 이 그래프를 뿌리에서 부터 잎까지 추적하면 chain rule에 따라 변화도를 자동으로 계산 

### 순전파 단계에서 autograd 작업 
1. 요청된 연산을 수행하여 결과 텐서를 계산 
2. DAG에 연산의 gradient function을 maintain 

### 역전파 단계에서 autograd 작업
- 뿌리에서 backword()가 호출될 때 시작 
1. 각 .grad_fn 으로부터 변화도를 계산
2. 각 텐서의 .grad 속성에 계산 결과를 쌓고(accumulate) 
3. chain rule을 사용하여, 모든 잎 텐서들까지 propagate 